In [1]:
import multiprocessing as mp
# import parmap
import sys, os
import pandas as pd
import gzip

# List of inputs
# ref = pd.read_csv(snakemake.input.ref, compression="gzip", sep="\t")
ref = pd.read_csv("/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/BCFTOOLS_CONCAT_TAB/LanexHGSVCpool2NEW/merge.txt.gz", compression="gzip", sep="\t")
ref

,ID,AC,AF,SAMPLE
0,10:49230:C:T,21,0.003279,HG00238
1,10:49248:G:A,17,0.002655,HG00238
2,10:49257:A:G,18,0.002811,HG00238
3,10:49264:G:C,17,0.002655,HG00238
4,10:49269:A:G,17,0.002655,HG00238
...,...,...,...,...
16837406,X:156021673:C:T,7,0.001093,NA21126
16837407,X:156021873:C:T,296,0.046221,NA21126
16837408,X:156023995:G:A,6,0.000937,NA21126
16837409,X:156028529:C:T,80,0.012492,NA21126


In [2]:

# def aggregate_sample_vcf(file):
#     sample = file.split("/")[-1].split(".")[0]
#     vcf = f"/g/korbel/weber/MosaiCatcher_files/EXTERNAL_DATA/snv_sites_to_genotype/1000G_SNV_with_GT/BCFTOOLS_OTF/{file}.vcf.gz"
#     skip = len([e.decode("ISO-8859–1").split('\n') for e in gzip.open("{}".format(vcf, "rb")) if e.decode("ISO-8859–1").split('\n')[0].startswith("##")])
#     df = pd.read_csv(vcf, compression="gzip", skiprows=skip, sep="\t")
#     df["#CHROM"] = df["#CHROM"].astype(str)
#     df["#CHROM"] = df["#CHROM"].str.replace("chr", "")
#     df["ID"] = df["#CHROM"].astype(str) + ":" + df["POS"].astype(str) + ":" + df["REF"] + ":" + df["ALT"]
#     df["GT"] = df[sample]
#     df = df.loc[(df["GT"] != "1/1") & (df["GT"] != "0/0")]
#     df = df.drop([sample], axis=1)
#     df["sample"] = sample
    
#     return df

# l = [
#     "HG00238",	"HG00342",	"HG00479",	"HG00623",	"HG00702",	"HG00705",	"HG00739",	"HG01800",	"HG02014",	"HG02155",	"HG02178",	"HG02190",	"HG02312",	"HG02485",	"HG02643",	"HG02679",	"HG02793",	"HG02944",	"HG03121",	"HG03133",	"HG03369",	"HG03371",	"HG03548",	"HG03673",	"HG03812",	"HG03882",	"HG03897",	"NA10861",	"NA18566",	"NA18879",	"NA18907",	"NA18913",	"NA18941",	"NA19451",	"NA19455",	"NA19788",	"NA20525",	"NA20861",	"NA21126",
# ]
# l_df = []
# for e in l:
#     print(e)
#     df = aggregate_sample_vcf(e)
#     l_df.append(df)

# l_df = pd.concat(l_df)
# l_df
# # df.to_csv("/g/korbel/weber/Christina/MosaiCatcher/EXTERNAL_DATA/1000G_SNV_with_GT/{}.vcf.gz".format(e), compression="gzip", sep="\t", index=False)


In [3]:
# vcf = "/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A06.vcf.gz"
# skip = len([e.decode("ISO-8859–1").split('\n') for e in gzip.open("{}".format(vcf, "rb")) if e.decode("ISO-8859–1").split('\n')[0].startswith("##")])
# df = pd.read_csv(vcf, compression="gzip", skiprows=skip, sep="\t")
# df["#CHROM"] = df["#CHROM"].astype(str)
# df["#CHROM"] = df["#CHROM"].str.replace("chr", "")
# df["ID"] = df["#CHROM"].astype(str) + ":" + df["POS"].astype(str) + ":" + df["REF"] + ":" + df["ALT"]

# df["LanexHGSVCpool2NEW"] = df["LanexHGSVCpool2NEW"].apply(lambda x: x.split(":")[0])

# df = df.loc[(df["LanexHGSVCpool2NEW"] != "1/1") & (df["LanexHGSVCpool2NEW"] != "0/0")]
# df
# # df


In [4]:
# merge_df = pd.merge(l_df[["ID", "sample"]], df[["ID", "LanexHGSVCpool2NEW"]], on="ID", how="inner")
# merge_df 

In [5]:
# merge_df = merge_df.loc[merge_df["sample"] == "HG03897"]
# merge_df 

In [6]:
# merge_df["sample"].value_counts()

In [7]:
# m = mp.Manager()
# l_df_gb = m.list()
# l_df = m.list()

l_df_gb = []
l_df = []

# def mp_vcf(vcf, l_df_gb, l_df):
def mp_vcf(vcf):

    # Nb of rows to skip at the beginning of the VCF
    skip = len([e.decode("ISO-8859–1").split('\n') for e in gzip.open("{}".format(vcf, "rb")) if e.decode("ISO-8859–1").split('\n')[0].startswith("##")])
    # chrom = vcf.split("/")[-3]
    sample = vcf.split("/")[-1].replace(".vcf.gz", "")
    df = pd.read_csv(vcf, compression="gzip", skiprows=skip, sep="\t")
    df["#CHROM"] = df["#CHROM"].astype(str)
    df["#CHROM"] = df["#CHROM"].str.replace("chr", "")
    df["ID"] = df["#CHROM"].astype(str) + ":" + df["POS"].astype(str) + ":" + df["REF"] + ":" + df["ALT"]

    # df = df.loc[df["QUAL"] >= 10]

    merge_df = pd.merge(ref, df, on="ID", how='inner')

    # merge_df["SAMPLE"] = merge_df["SAMPLE"].fillna("NAN")
    merge_df["QUERY_SAMPLE_CELL"] = sample
    

    merge_df_gb = merge_df.groupby("SAMPLE")["ID"].nunique().sort_values(ascending=False).reset_index()
    merge_df_gb["QUERY_SAMPLE_CELL"] = sample
    merge_df_gb["Rank"] = list(range(1,1+merge_df_gb.shape[0]))
    merge_df_gb["Total_SNP"] = df.shape[0]

    l_df_gb.append(merge_df_gb)
    l_df.append(merge_df)
# f = snakemake.input.gt_sample_folder
f = "/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW"
f_ldir = sorted([f"{f}/{e}" for e in os.listdir(f) if e.endswith(".vcf.gz")])
# f_ldir = ["/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A06.vcf.gz"]
print(f_ldir)
# f_ldir = sorted(list(snakemake.input.vcf))
# parmap.starmap(mp_vcf, list(zip(f_ldir)), l_df_gb, l_df)

skip = len([e.decode("ISO-8859–1").split('\n') for e in gzip.open("{}".format(f_ldir[0], "rb")) if e.decode("ISO-8859–1").split('\n')[0].startswith("##")])

# df = pd.read_csv(f_ldir[0], compression="gzip", skiprows=skip, sep="\t")
# df["#CHROM"] = df["#CHROM"].astype(str)
# df["#CHROM"] = df["#CHROM"].str.replace("chr", "")
# df["ID"] = df["#CHROM"].astype(str) + ":" + df["POS"].astype(str) + ":" + df["REF"] + ":" + df["ALT"]




for e in f_ldir:
    print(e)
    mp_vcf(e)



['/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A01.vcf.gz', '/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A02.vcf.gz', '/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A03.vcf.gz', '/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A04.vcf.gz', '/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A05.vcf.gz', '/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A06.vcf.gz', '/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A07.vcf.gz', '/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/GENOTYPING_OTF/LanexHGSVCpool2NEW/LanexHGSVCpool2NEWiTRU1A08.vcf.gz', '/scratch/tweber/DATA/MC_DATA/D

In [8]:
# df["GT"] = df["LanexHGSVCpool2NEW"].str.split(":", expand=True)[0]
# df.loc[(df["GT"] != "1/1") & (df["GT"] != "0/0")]

In [9]:
pd.options.display.max_rows = 250
concat_df = pd.concat(list(l_df))
concat_df["GT"] = concat_df["LanexHGSVCpool2NEW"].str.split(":", expand=True)[0]
concat_df = concat_df.sort_values(by=["ID"])
concat_df["QUERY_SAMPLE_CELL"] = concat_df["QUERY_SAMPLE_CELL"].apply(lambda x: x.split("/")[-1].replace(".vcf.gz", ""))
# concat_df.loc[(concat_df["GT"] != "1/1") & (concat_df["GT"] != "0/0")]
concat_df

,ID,AC,AF,SAMPLE,#CHROM,POS,REF,ALT,QUAL,FILTER,INFO,FORMAT,LanexHGSVCpool2NEW,QUERY_SAMPLE_CELL,GT
90,10:100244326:C:T,19,0.002967,HG03897,10,100244326,C,T,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE5E46,1/1
89,10:100244326:C:T,19,0.002967,HG02793,10,100244326,C,T,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE5E46,1/1
90,10:100327175:G:C,4,0.000625,HG02190,10,100327175,G,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRU1A07,1/1
179,10:100342595:G:A,227,0.035447,NA20525,10,100342595,G,A,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE7G09,1/1
693,10:100372048:T:C,109,0.017021,HG03133,10,100372048,T,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE5E30,1/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,X:99653864:C:T,183,0.038077,HG02679,X,99653864,C,T,41.1451,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:31.5404:2:0,2:0:0:2:56",LanexHGSVCpool2NEWiTRUE7G61,1/1
705,X:9968565:G:C,68,0.014149,HG03369,X,9968565,G,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRU1A17,1/1
704,X:9968565:G:C,68,0.014149,HG02643,X,9968565,G,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRU1A17,1/1
80,X:99969663:G:C,64,0.013317,HG02485,X,99969663,G,C,52.5495,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:25.0573:2:0,2:0:0:2:68",LanexHGSVCpool2NEWiTRUE7G50,1/1


In [10]:

# concat_counts_df = concat_df.loc[concat_df["QUAL"] >= 10].groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()

concat_counts_df = concat_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()
concat_counts_df["Rank"] = concat_counts_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].transform(lambda r: range(1, len(r) + 1))
concat_counts_df

,QUERY_SAMPLE_CELL,SAMPLE,Counts,Rank
0,LanexHGSVCpool2NEWiTRU1A02,NA19788,12,1
1,LanexHGSVCpool2NEWiTRU1A02,HG02312,7,2
2,LanexHGSVCpool2NEWiTRU1A02,HG03882,6,3
3,LanexHGSVCpool2NEWiTRU1A02,NA18941,6,4
4,LanexHGSVCpool2NEWiTRU1A02,HG00702,5,5
...,...,...,...,...
12935,LanexHGSVCpool2NEWiTRUE7G95,NA18941,1,27
12936,LanexHGSVCpool2NEWiTRUE7G95,HG02643,1,28
12937,LanexHGSVCpool2NEWiTRUE7G95,HG02485,1,29
12938,LanexHGSVCpool2NEWiTRUE7G95,HG00479,1,30


In [11]:
# coverage_df = pd.read_csv(snakemake.input.coverage, sep="\t")
coverage_df = pd.read_csv("/scratch/tweber/DATA/MC_DATA/DEMULTIPLEXING_POOLS/COVERAGE/LanexHGSVCpool2NEW/MERGE/merge_coverage.txt", sep="\t")
# coverage_df = pd.read_csv("/scratch/tweber/DATA/1000G_SNV_with_GT/OTF/COVERAGE/HGSVCxPool1/MERGE/merge_coverage.txt", sep="\t")
# coverage_df.describe()
coverage_df

,QUERY_SAMPLE_CELL,meandepth
0,LanexHGSVCpool2NEWiTRU1A01,2.517345e-05
1,LanexHGSVCpool2NEWiTRU1A02,1.013690e-02
2,LanexHGSVCpool2NEWiTRU1A03,1.591056e-02
3,LanexHGSVCpool2NEWiTRU1A04,1.132580e-02
4,LanexHGSVCpool2NEWiTRU1A05,1.072678e-02
...,...,...
379,LanexHGSVCpool2NEWiTRUE7G92,5.509408e-03
380,LanexHGSVCpool2NEWiTRUE7G93,7.366145e-03
381,LanexHGSVCpool2NEWiTRUE7G94,1.136901e-02
382,LanexHGSVCpool2NEWiTRUE7G95,8.935367e-03


In [12]:
ashleys_predictions = pd.read_csv("/scratch/tweber/DATA/MC_DATA/STOCKS/2023-11-09-HW3YVAFX5/LanexHGSVCpool2NEW/cell_selection/labels.tsv", sep="\t").rename({"cell": "QUERY_SAMPLE_CELL", "prediction" : "ashleys_prediction", "probability": "ashleys_probability"}, axis=1).sort_values(by="QUERY_SAMPLE_CELL")
# ashleys_predictions = pd.read_csv(snakemake.input.ashleys_predictions, sep="\t").rename({"cell": "QUERY_SAMPLE_CELL", "prediction" : "ashleys_prediction", "probability": "ashleys_probability"}, axis=1).sort_values(by="QUERY_SAMPLE_CELL")
# ashleys_predictions = pd.read_csv("/g/korbel2/weber/MosaiCatcher_files/POOLING/POOL1_RESEQ/HGSVCxPool1/cell_selection/labels.tsv", sep="\t").rename({"cell": "QUERY_SAMPLE_CELL", "prediction" : "ashleys_prediction", "probability": "ashleys_probability"}, axis=1).sort_values(by="QUERY_SAMPLE_CELL")
ashleys_predictions["QUERY_SAMPLE_CELL"] = ashleys_predictions["QUERY_SAMPLE_CELL"].str.replace(".sort.mdup.bam", "")
# mc_predictions = pd.read_csv("/g/korbel2/weber/MosaiCatcher_files/POOLING/POOL1_RESEQ/HGSVCxPool1/counts/HGSVCxPool1.info_raw", skiprows=13, sep="\t").rename({"cell": "QUERY_SAMPLE_CELL"}, axis=1)[["QUERY_SAMPLE_CELL", "mapped", "good", "pass1"]].rename({"mapped": "reads_mapped", "good" : "reads_used", "pass1": "mc_coverage_compliant"}, axis=1)
mc_predictions = pd.read_csv("/scratch/tweber/DATA/MC_DATA/STOCKS/2023-11-09-HW3YVAFX5/LanexHGSVCpool2NEW/counts/LanexHGSVCpool2NEW.info_raw", skiprows=13, sep="\t").rename({"cell": "QUERY_SAMPLE_CELL"}, axis=1)[["QUERY_SAMPLE_CELL", "mapped", "good", "pass1"]].rename({"mapped": "reads_mapped", "good" : "reads_used", "pass1": "mc_coverage_compliant"}, axis=1)
# mc_predictions = pd.read_csv(snakemake.input.mc_predictions, skiprows=13, sep="\t").rename({"cell": "QUERY_SAMPLE_CELL"}, axis=1)[["QUERY_SAMPLE_CELL", "mapped", "good", "pass1"]].rename({"mapped": "reads_mapped", "good" : "reads_used", "pass1": "mc_coverage_compliant"}, axis=1)

ashleys_mc_predictions = pd.merge(
    ashleys_predictions,
    mc_predictions,
    on="QUERY_SAMPLE_CELL"
)
ashleys_mc_predictions.loc[(ashleys_mc_predictions["ashleys_prediction"] == 1) & (ashleys_mc_predictions["mc_coverage_compliant"] == 1), "Used/Not used in MC"] = 1
ashleys_mc_predictions["Used/Not used in MC"] = ashleys_mc_predictions["Used/Not used in MC"].fillna(0)
ashleys_mc_predictions["Used/Not used in MC"] = ashleys_mc_predictions["Used/Not used in MC"].astype(int)
ashleys_mc_predictions

,QUERY_SAMPLE_CELL,ashleys_prediction,ashleys_probability,sample,reads_mapped,reads_used,mc_coverage_compliant,Used/Not used in MC
0,LanexHGSVCpool2NEWiTRU1A01,0,0.0000,LanexHGSVCpool2NEW,7689,570,0,0
1,LanexHGSVCpool2NEWiTRU1A02,0,0.0238,LanexHGSVCpool2NEW,752775,191571,1,0
2,LanexHGSVCpool2NEWiTRU1A03,1,0.7255,LanexHGSVCpool2NEW,1239725,296211,1,1
3,LanexHGSVCpool2NEWiTRU1A04,1,0.8534,LanexHGSVCpool2NEW,805153,213334,1,1
4,LanexHGSVCpool2NEWiTRU1A05,1,0.8491,LanexHGSVCpool2NEW,740280,199022,1,1
...,...,...,...,...,...,...,...,...
379,LanexHGSVCpool2NEWiTRUE7G92,0,0.3734,LanexHGSVCpool2NEW,299498,101620,1,0
380,LanexHGSVCpool2NEWiTRUE7G93,1,0.6969,LanexHGSVCpool2NEW,416741,138447,1,1
381,LanexHGSVCpool2NEWiTRUE7G94,1,0.8808,LanexHGSVCpool2NEW,702895,210079,1,1
382,LanexHGSVCpool2NEWiTRUE7G95,1,0.8923,LanexHGSVCpool2NEW,506203,165985,1,1


In [13]:
concat_df

,ID,AC,AF,SAMPLE,#CHROM,POS,REF,ALT,QUAL,FILTER,INFO,FORMAT,LanexHGSVCpool2NEW,QUERY_SAMPLE_CELL,GT
90,10:100244326:C:T,19,0.002967,HG03897,10,100244326,C,T,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE5E46,1/1
89,10:100244326:C:T,19,0.002967,HG02793,10,100244326,C,T,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE5E46,1/1
90,10:100327175:G:C,4,0.000625,HG02190,10,100327175,G,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRU1A07,1/1
179,10:100342595:G:A,227,0.035447,NA20525,10,100342595,G,A,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE7G09,1/1
693,10:100372048:T:C,109,0.017021,HG03133,10,100372048,T,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRUE5E30,1/1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,X:99653864:C:T,183,0.038077,HG02679,X,99653864,C,T,41.1451,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:31.5404:2:0,2:0:0:2:56",LanexHGSVCpool2NEWiTRUE7G61,1/1
705,X:9968565:G:C,68,0.014149,HG03369,X,9968565,G,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRU1A17,1/1
704,X:9968565:G:C,68,0.014149,HG02643,X,9968565,G,C,56.3287,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:32.0278:2:0,2:0:0:2:72",LanexHGSVCpool2NEWiTRU1A17,1/1
80,X:99969663:G:C,64,0.013317,HG02485,X,99969663,G,C,52.5495,.,AB=0;ABP=0;AC=2;AF=1;AN=2;AO=2;CIGAR=1X;DP=2;D...,GT:GQ:DP:AD:RO:QR:AO:QA,"1/1:25.0573:2:0,2:0:0:2:68",LanexHGSVCpool2NEWiTRUE7G50,1/1


In [14]:
qual_cutoff = 10
merge_df = pd.merge(
        concat_df.groupby("QUERY_SAMPLE_CELL")["ID"].nunique().rename("QUAL >= 0").reset_index(),
        concat_df.loc[concat_df["QUAL"] >= qual_cutoff].groupby("QUERY_SAMPLE_CELL")["ID"].nunique().rename("QUAL >= {}".format(str(qual_cutoff))).reset_index(),
        on="QUERY_SAMPLE_CELL",how="outer").sort_values(by="QUERY_SAMPLE_CELL")
merge_df

,QUERY_SAMPLE_CELL,QUAL >= 0,QUAL >= 10
0,LanexHGSVCpool2NEWiTRU1A02,44,43
1,LanexHGSVCpool2NEWiTRU1A03,82,77
2,LanexHGSVCpool2NEWiTRU1A04,46,43
3,LanexHGSVCpool2NEWiTRU1A05,40,39
4,LanexHGSVCpool2NEWiTRU1A06,95,88
...,...,...,...
361,LanexHGSVCpool2NEWiTRUE7G91,213,206
362,LanexHGSVCpool2NEWiTRUE7G92,109,108
363,LanexHGSVCpool2NEWiTRUE7G93,133,132
364,LanexHGSVCpool2NEWiTRUE7G94,272,267


In [15]:

# import numpy as np
# vcf_list = list(concat_df.QUERY_SAMPLE_CELL.unique())
# empty_samples_list = [{"QUERY_SAMPLE_CELL" : "HGSVCPool1x02PE20{}".format(str(e)), "QUAL >= 0" : np.nan, "QUAL >= {}".format(str(qual_cutoff)) : np.nan, } for e in list(range(301, 397)) if "HGSVCPool1x02PE20{}".format(str(e)) not in vcf_list]

# merge_df = pd.concat([merge_df, pd.DataFrame(empty_samples_list)]).sort_values(by="QUERY_SAMPLE_CELL")
# merge_df

In [16]:



concat_counts_df = concat_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()
concat_counts_df["Rank"] = concat_counts_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].transform(lambda r: range(1, len(r) + 1))
concat_counts_df


,QUERY_SAMPLE_CELL,SAMPLE,Counts,Rank
0,LanexHGSVCpool2NEWiTRU1A02,NA19788,12,1
1,LanexHGSVCpool2NEWiTRU1A02,HG02312,7,2
2,LanexHGSVCpool2NEWiTRU1A02,HG03882,6,3
3,LanexHGSVCpool2NEWiTRU1A02,NA18941,6,4
4,LanexHGSVCpool2NEWiTRU1A02,HG00702,5,5
...,...,...,...,...
12935,LanexHGSVCpool2NEWiTRUE7G95,NA18941,1,27
12936,LanexHGSVCpool2NEWiTRUE7G95,HG02643,1,28
12937,LanexHGSVCpool2NEWiTRUE7G95,HG02485,1,29
12938,LanexHGSVCpool2NEWiTRUE7G95,HG00479,1,30


In [17]:
concat_counts_df.loc[concat_counts_df["Rank"] <= 3]

,QUERY_SAMPLE_CELL,SAMPLE,Counts,Rank
0,LanexHGSVCpool2NEWiTRU1A02,NA19788,12,1
1,LanexHGSVCpool2NEWiTRU1A02,HG02312,7,2
2,LanexHGSVCpool2NEWiTRU1A02,HG03882,6,3
27,LanexHGSVCpool2NEWiTRU1A03,HG02312,11,1
28,LanexHGSVCpool2NEWiTRU1A03,NA19788,11,2
...,...,...,...,...
12876,LanexHGSVCpool2NEWiTRUE7G94,HG03371,56,2
12877,LanexHGSVCpool2NEWiTRUE7G94,HG02944,51,3
12909,LanexHGSVCpool2NEWiTRUE7G95,HG03812,7,1
12910,LanexHGSVCpool2NEWiTRUE7G95,HG00238,6,2


In [18]:
# test = pd.pivot_table(concat_counts_df.loc[concat_counts_df["Rank"] <= 3], index="QUERY_SAMPLE_CELL", columns=["Rank"], values=["SAMPLE"], aggfunc=lambda x: ' '.join(x))
# test.columns = test.columns.droplevel()
# test

In [19]:
merge_df.columns

Index(['QUERY_SAMPLE_CELL', 'QUAL >= 0', 'QUAL >= 10'], dtype='object')

In [20]:
# First pivot table
pivot_table1 = pd.pivot_table(
    concat_counts_df.loc[concat_counts_df["Rank"] <= 3], 
    index="QUERY_SAMPLE_CELL", 
    columns=["Rank"], 
    values=["SAMPLE"], 
    aggfunc=lambda x: ' '.join(x)
).reset_index()
pivot_table1.columns = ['_'.join(map(str, col)).strip() for col in pivot_table1.columns.values]
pivot_table1.columns = ["QUERY_SAMPLE_CELL", "SAMPLE_1", "SAMPLE_2", "SAMPLE_3"]
pivot_table1


,QUERY_SAMPLE_CELL,SAMPLE_1,SAMPLE_2,SAMPLE_3
0,LanexHGSVCpool2NEWiTRU1A02,NA19788,HG02312,HG03882
1,LanexHGSVCpool2NEWiTRU1A03,HG02312,NA19788,NA20861
2,LanexHGSVCpool2NEWiTRU1A04,HG00342,NA10861,NA20525
3,LanexHGSVCpool2NEWiTRU1A05,NA20861,HG02793,NA21126
4,LanexHGSVCpool2NEWiTRU1A06,HG03897,HG00238,HG02793
...,...,...,...,...
361,LanexHGSVCpool2NEWiTRUE7G91,HG03548,HG02643,HG03121
362,LanexHGSVCpool2NEWiTRUE7G92,HG03121,HG02643,HG03548
363,LanexHGSVCpool2NEWiTRUE7G93,HG02679,HG03369,HG03371
364,LanexHGSVCpool2NEWiTRUE7G94,NA18879,HG03371,HG02944


In [21]:
# Merge with the first pivot table
merge_df = pd.merge(
    merge_df,
    pivot_table1,
    on="QUERY_SAMPLE_CELL",
    how="left"
)
merge_df


,QUERY_SAMPLE_CELL,QUAL >= 0,QUAL >= 10,SAMPLE_1,SAMPLE_2,SAMPLE_3
0,LanexHGSVCpool2NEWiTRU1A02,44,43,NA19788,HG02312,HG03882
1,LanexHGSVCpool2NEWiTRU1A03,82,77,HG02312,NA19788,NA20861
2,LanexHGSVCpool2NEWiTRU1A04,46,43,HG00342,NA10861,NA20525
3,LanexHGSVCpool2NEWiTRU1A05,40,39,NA20861,HG02793,NA21126
4,LanexHGSVCpool2NEWiTRU1A06,95,88,HG03897,HG00238,HG02793
...,...,...,...,...,...,...
361,LanexHGSVCpool2NEWiTRUE7G91,213,206,HG03548,HG02643,HG03121
362,LanexHGSVCpool2NEWiTRUE7G92,109,108,HG03121,HG02643,HG03548
363,LanexHGSVCpool2NEWiTRUE7G93,133,132,HG02679,HG03369,HG03371
364,LanexHGSVCpool2NEWiTRUE7G94,272,267,NA18879,HG03371,HG02944


In [22]:
concat_counts_df.loc[concat_counts_df["Rank"] <= 3]


,QUERY_SAMPLE_CELL,SAMPLE,Counts,Rank
0,LanexHGSVCpool2NEWiTRU1A02,NA19788,12,1
1,LanexHGSVCpool2NEWiTRU1A02,HG02312,7,2
2,LanexHGSVCpool2NEWiTRU1A02,HG03882,6,3
27,LanexHGSVCpool2NEWiTRU1A03,HG02312,11,1
28,LanexHGSVCpool2NEWiTRU1A03,NA19788,11,2
...,...,...,...,...
12876,LanexHGSVCpool2NEWiTRUE7G94,HG03371,56,2
12877,LanexHGSVCpool2NEWiTRUE7G94,HG02944,51,3
12909,LanexHGSVCpool2NEWiTRUE7G95,HG03812,7,1
12910,LanexHGSVCpool2NEWiTRUE7G95,HG00238,6,2


In [23]:
pivot_table2 = pd.pivot_table(
    concat_counts_df.loc[concat_counts_df["Rank"] <= 3, ["QUERY_SAMPLE_CELL", "SAMPLE", "Counts", "Rank"]], 
    index="QUERY_SAMPLE_CELL", 
    columns=["Rank"],
    values=["Counts"]
).reset_index()
pivot_table2.columns = ['_'.join(map(str, col)).strip() for col in pivot_table2.columns.values]
pivot_table2.columns = ["QUERY_SAMPLE_CELL", "SAMPLE_1_SNP_nb", "SAMPLE_2_SNP_nb", "SAMPLE_3_SNP_nb"]
pivot_table2


,QUERY_SAMPLE_CELL,SAMPLE_1_SNP_nb,SAMPLE_2_SNP_nb,SAMPLE_3_SNP_nb
0,LanexHGSVCpool2NEWiTRU1A02,12,7,6
1,LanexHGSVCpool2NEWiTRU1A03,11,11,7
2,LanexHGSVCpool2NEWiTRU1A04,10,7,5
3,LanexHGSVCpool2NEWiTRU1A05,7,7,5
4,LanexHGSVCpool2NEWiTRU1A06,18,13,13
...,...,...,...,...
361,LanexHGSVCpool2NEWiTRUE7G91,49,40,37
362,LanexHGSVCpool2NEWiTRUE7G92,25,22,18
363,LanexHGSVCpool2NEWiTRUE7G93,29,26,24
364,LanexHGSVCpool2NEWiTRUE7G94,63,56,51


In [24]:
merge_df = pd.merge(
    merge_df,
    pivot_table2,
    on="QUERY_SAMPLE_CELL",
    how="left"
)
merge_df

,QUERY_SAMPLE_CELL,QUAL >= 0,QUAL >= 10,SAMPLE_1,SAMPLE_2,SAMPLE_3,SAMPLE_1_SNP_nb,SAMPLE_2_SNP_nb,SAMPLE_3_SNP_nb
0,LanexHGSVCpool2NEWiTRU1A02,44,43,NA19788,HG02312,HG03882,12,7,6
1,LanexHGSVCpool2NEWiTRU1A03,82,77,HG02312,NA19788,NA20861,11,11,7
2,LanexHGSVCpool2NEWiTRU1A04,46,43,HG00342,NA10861,NA20525,10,7,5
3,LanexHGSVCpool2NEWiTRU1A05,40,39,NA20861,HG02793,NA21126,7,7,5
4,LanexHGSVCpool2NEWiTRU1A06,95,88,HG03897,HG00238,HG02793,18,13,13
...,...,...,...,...,...,...,...,...,...
361,LanexHGSVCpool2NEWiTRUE7G91,213,206,HG03548,HG02643,HG03121,49,40,37
362,LanexHGSVCpool2NEWiTRUE7G92,109,108,HG03121,HG02643,HG03548,25,22,18
363,LanexHGSVCpool2NEWiTRUE7G93,133,132,HG02679,HG03369,HG03371,29,26,24
364,LanexHGSVCpool2NEWiTRUE7G94,272,267,NA18879,HG03371,HG02944,63,56,51


In [25]:



# concat_counts_df = concat_df.loc[concat_df["QUAL"] >= qual_cutoff].groupby("QUERY_SAMPLE_CELL")["SAMPLE"].value_counts().rename("Counts").reset_index()
# concat_counts_df["Rank"] = concat_counts_df.groupby("QUERY_SAMPLE_CELL")["SAMPLE"].transform(lambda r: range(1, len(r) + 1))

# merge_df = pd.merge(
#         merge_df,
#         pd.pivot_table(concat_counts_df.loc[concat_counts_df["Rank"] <= 3], index="QUERY_SAMPLE_CELL", columns=["Rank"], values=["SAMPLE"], aggfunc=lambda x: ' '.join(x)),
#         on="QUERY_SAMPLE_CELL",
#         how="left"
# )
# merge_df = pd.merge(
#         merge_df,
#         pd.pivot_table(concat_counts_df.loc[concat_counts_df["Rank"] <= 3], index="QUERY_SAMPLE_CELL", columns=["Rank"]),
#         on="QUERY_SAMPLE_CELL",
#         how="left"
# )
merge_df = pd.merge(
        merge_df,
        coverage_df,
        on="QUERY_SAMPLE_CELL",
        how="left"
)
merge_df = pd.merge(
        merge_df,
        ashleys_mc_predictions,
        on="QUERY_SAMPLE_CELL",
        how="left"
)

# pd.options.display.max_columns = 50
# merge_df.columns = ['QUERY_SAMPLE_CELL', 'QUAL>=0', 'QUAL>={}'.format(str(qual_cutoff)), 'QUAL>=0_SAMPLE_1', 'QUAL>=0_SAMPLE_2', 'QUAL>=0_SAMPLE_3', 'QUAL>=0_Counts_1', 'QUAL>=0_Counts_2', 'QUAL>=0_Counts_3',
# 'QUAL>={}_SAMPLE_1'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_2'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_3'.format(str(qual_cutoff)), 'QUAL>={}_Counts_1'.format(str(qual_cutoff)), 'QUAL>={}_Counts_2'.format(str(qual_cutoff)), 'QUAL>={}_Counts_3'.format(str(qual_cutoff)),  'Average_Coverage', 'ashleys_prediction', 'ashleys_probability', 'reads_mapped', 'reads_used', 'mc_coverage_compliant', 'Used/Not used in MC']


# # merge_df = merge_df[
# #         [
# #              'QUERY_SAMPLE_CELL',   'QUAL>={}'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_1'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_2'.format(str(qual_cutoff)), 'QUAL>={}_SAMPLE_3'.format(str(qual_cutoff)), 'QUAL>={}_Counts_1'.format(str(qual_cutoff)), 'QUAL>={}_Counts_2'.format(str(qual_cutoff)), 'QUAL>={}_Counts_3'.format(str(qual_cutoff)),  'Average_Coverage', 'ashleys_prediction', 'ashleys_probability', 'reads_mapped', 'reads_used', 'mc_coverage_compliant', 'Used/Not used in MC'
# #         ]
# # ]
# merge_df["Average_Coverage"] = merge_df["Average_Coverage"]*100

# # merge_df = merge_df.loc[merge_df["mc_coverage_compliant"] == 1].sort_values(by="Average_Coverage")

merge_df.to_excel("LanexHGSVCpool2NEW.xlsx", index=False)
merge_df

# # merge_df.to_excel(snakemake.output.stats, index=False)


# # import os, sys, stat
# # os.chmod("/g/korbel2/weber/MosaiCatcher_files/POOLING/POOL1_RESEQ/HGSVCxPool1/config/stats_pooling.xlsx", stat.S_IRWXG)

# merge_df.sort_values(by="Average_Coverage")


,QUERY_SAMPLE_CELL,QUAL >= 0,QUAL >= 10,SAMPLE_1,SAMPLE_2,SAMPLE_3,SAMPLE_1_SNP_nb,SAMPLE_2_SNP_nb,SAMPLE_3_SNP_nb,meandepth,ashleys_prediction,ashleys_probability,sample,reads_mapped,reads_used,mc_coverage_compliant,Used/Not used in MC
0,LanexHGSVCpool2NEWiTRU1A02,44,43,NA19788,HG02312,HG03882,12,7,6,0.010137,0,0.0238,LanexHGSVCpool2NEW,752775,191571,1,0
1,LanexHGSVCpool2NEWiTRU1A03,82,77,HG02312,NA19788,NA20861,11,11,7,0.015911,1,0.7255,LanexHGSVCpool2NEW,1239725,296211,1,1
2,LanexHGSVCpool2NEWiTRU1A04,46,43,HG00342,NA10861,NA20525,10,7,5,0.011326,1,0.8534,LanexHGSVCpool2NEW,805153,213334,1,1
3,LanexHGSVCpool2NEWiTRU1A05,40,39,NA20861,HG02793,NA21126,7,7,5,0.010727,1,0.8491,LanexHGSVCpool2NEW,740280,199022,1,1
4,LanexHGSVCpool2NEWiTRU1A06,95,88,HG03897,HG00238,HG02793,18,13,13,0.017471,1,0.8890,LanexHGSVCpool2NEW,1244483,320990,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,LanexHGSVCpool2NEWiTRUE7G91,213,206,HG03548,HG02643,HG03121,49,40,37,0.011471,1,0.7426,LanexHGSVCpool2NEW,824535,212520,1,1
362,LanexHGSVCpool2NEWiTRUE7G92,109,108,HG03121,HG02643,HG03548,25,22,18,0.005509,0,0.3734,LanexHGSVCpool2NEW,299498,101620,1,0
363,LanexHGSVCpool2NEWiTRUE7G93,133,132,HG02679,HG03369,HG03371,29,26,24,0.007366,1,0.6969,LanexHGSVCpool2NEW,416741,138447,1,1
364,LanexHGSVCpool2NEWiTRUE7G94,272,267,NA18879,HG03371,HG02944,63,56,51,0.011369,1,0.8808,LanexHGSVCpool2NEW,702895,210079,1,1


In [26]:
# merge_df[['ashleys_prediction', 'mc_coverage_compliant', 'Used/Not used in MC']].sum()

In [27]:
# import seaborn as sns
# import matplotlib.pyplot as plt

# from matplotlib.colors import LogNorm, Normalize


# plt.style.use('default')
# plt.figure(figsize=(10,20))


# merge_df

# ax = sns.barplot(data=merge_df.sort_values(by="QUERY_SAMPLE_CELL"),  y="QUERY_SAMPLE_CELL", x="Average_Coverage", hue="mc_coverage_compliant")
# ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
# ax.axvline(0.34, ls="--", color="grey", lw=0.5)
# ax.set_xlabel("Average Coverage %")

In [28]:
merge_df.columns

Index(['QUERY_SAMPLE_CELL', 'QUAL >= 0', 'QUAL >= 10', 'SAMPLE_1', 'SAMPLE_2',
       'SAMPLE_3', 'SAMPLE_1_SNP_nb', 'SAMPLE_2_SNP_nb', 'SAMPLE_3_SNP_nb',
       'meandepth', 'ashleys_prediction', 'ashleys_probability', 'sample',
       'reads_mapped', 'reads_used', 'mc_coverage_compliant',
       'Used/Not used in MC'],
      dtype='object')